This is an introductory notebook exploring building a simple neural network via PyTorch for the CIFAR-10 dataset.

In [ ]:
import torch #pytorch package
import torchvision #package that deals with datasets and pretrained neural nets
import torch.utils.data
import torchvision.transforms as transforms

In [ ]:
# defining a transformation of images to torch tensors, using transforms.ToTensor()
# defining the values for standardizing the images, the mean and standard deviation for each channel using pre-computed numbers

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize( (0.4914, 0.48216, 0.44653), (0.24703, 0.24349, 0.26159) )]) #3 channels because RGB, Normalize(Mean, STD)

After importing the necessary packages and defining a function to transform the image dataset to tensors, we now import the CIFAR10 dataset as trainset and testset tensors using the transform function from the previous cell.


In [ ]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download = True, transform = transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download = True, transform = transform)


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


We now build trainloader and testloader to get the data ready for PyTorch. We set the minibatch to 32  to use 32  images per iteration (due to ds size). 

In [ ]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size = 32, shuffle = True, num_workers = 2)
testloader = torch.utils.data.DataLoader(testset, batch_size = 32, shuffle = False, num_workers= 2)

Now we can inspect the dataloader

In [ ]:
# print(testloader.dataset.test_data.shape, trainloader.dataset.train_data.shape)

print("Shape of test data: ", testloader.dataset.data.shape, )
print("Batch size of test data: ",testloader.batch_size)
print("Shape of train data: ", trainloader.dataset.data.shape, )
print("Batch size of train data: ",trainloader.batch_size)

Shape of test data:  (10000, 32, 32, 3)
Batch size of test data:  32
Shape of train data:  (50000, 32, 32, 3)
Batch size of train data:  32


Now we write the class for the neural network

In [ ]:
import torch.nn as nn
import torch.nn.functional as F #use this to apply ReLU non linearity in the hidden layer
import torch.optim as optim

class Net(nn.Module):
  def __init__(self):
    super(Net,self).__init__()
    self.fc1 = nn.Linear(32*32*3, 500) #image shape (32,32,3) // we set the no. of units in this hidden layer to be 500
    self.fc2 = nn.Linear(500,10)

  def forward(self, x):
    x = F.relu(self.fc1(x))
    return self.fc2(x)

We then train the actual neural network. First, we initiate the net, the loss (cross-entropy) and the optimizer (Adam). Adam optimizer works very well + is a version of gradient descent.

In [ ]:

net = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=3e-4)

for epoch in range(10): #loop over the dataset multiple times
  for i,data in enumerate(trainloader,0):
    #get the inputs
    inputs, labels = data
    inputs = inputs.view(-1, 32*32*3) #converts all the entries of the images into vectors

    # Zero the parameter gradients - in order to not accumulate gradients from the previous iterations
    optimizer.zero_grad()

    #Forward + backward + optimize
    outputs = net(inputs)
    loss = criterion(outputs, labels)
    loss.backward() #compute the gradient using loss.backward()
    optimizer.step() #changing the weights using our optimizer with the optimizer.step() command

After training the neural network, we then try and use the it to make predictions from our test data.

In [ ]:
correct, total = 0,0
predictions = []
net.eval()

for i, data in enumerate(testloader,0):
  inputs,labels = data
  inputs = inputs.view(-1, 32*32*3)
  outputs = net(inputs)
  _, predicted = torch.max(outputs.data,1) #get the class with the highest score using the max function
  predictions.append(outputs)
  total += labels.size(0)
  correct += (predicted == labels).sum().item()

print('The testing set accuracy of the network is %d %%' % (100 * correct/total)) #compute for the accuracy of your classifier